## Explore YOLO output

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import shutil
import sys
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    
root = find_project_root()

In [3]:
cam1_df = pd.read_csv(f"{root}/data/camera_1.csv")
cam2_df = pd.read_csv(f"{root}/data/camera_2.csv")

In [4]:
def print_df_info(df1, df2):
    # total records
    print(f"Total records in cam1: {len(df1)}")
    print(f"Total records in cam2: {len(df2)}")
    # total unique frames
    print(f"Total unique frames in cam1: {df1['frame_no'].nunique()}")
    print(f"Total unique frames in cam2: {df2['frame_no'].nunique()}")

print_df_info(cam1_df, cam2_df)

Total records in cam1: 15391
Total records in cam2: 4564
Total unique frames in cam1: 15391
Total unique frames in cam2: 4564


In [5]:
# sort by frame_no
cam1_df = cam1_df.sort_values(by='frame_no')
cam2_df = cam2_df.sort_values(by='frame_no')


# print first 5 rows of each dataframe
print(cam1_df.head())
print(cam2_df.head())

      x     y    timestamp  frame_no  confidence detection_method  camera
0  2567  1050  1550.000000      94.0    0.425080             YOLO       1
1  2569  1050  1566.666667      95.0    0.502407             YOLO       1
2  2573  1049  1583.333333      96.0    0.488850             YOLO       1
3  2575  1048  1600.000000      97.0    0.492749             YOLO       1
4  2578  1046  1616.666667      98.0    0.375369             YOLO       1
     x    y     timestamp  frame_no  confidence detection_method  camera
0  484  457  15533.333333     467.0    0.452926             YOLO       2
1  251  427  15566.666667     468.0    0.483836             YOLO       2
2  297  429  15600.000000     469.0    0.400000               LK       2
3  297  429  15633.333333     470.0    0.300000               LK       2
4  297  429  15666.666667     471.0    0.200000               LK       2


In [40]:
cam1_element = cam1_df.iloc[1].to_dict() # timestamp 1566.666667 
print(cam1_element)

cam2_element = cam2_df.iloc[1].to_dict() # timestamp 15566.666667
print(cam2_element)



{'x': 2569, 'y': 1050, 'timestamp': 1566.666666666667, 'frame_no': 95.0, 'confidence': 0.5024073123931885, 'detection_method': 'YOLO', 'camera': 1}
{'x': 251, 'y': 427, 'timestamp': 15566.666666666666, 'frame_no': 468.0, 'confidence': 0.4838364422321319, 'detection_method': 'YOLO', 'camera': 2}


In [7]:
def load_stereo_a_maps():
    file_path = f"{root}/output/STEREO_A_rectification_params.xml"
    cv_file = cv2.FileStorage(file_path, cv2.FILE_STORAGE_READ)

    stereo_maps = {
        "CAM_1_map_x": cv_file.getNode("CAM_1_map_x").mat(),
        "CAM_1_map_y": cv_file.getNode("CAM_1_map_y").mat(),
        "CAM_2_map_x": cv_file.getNode("CAM_2_map_x").mat(),
        "CAM_2_map_y": cv_file.getNode("CAM_2_map_y").mat(),
        "CAM_1_projection_matrix": cv_file.getNode("CAM_1_projection_matrix").mat(),
        "CAM_2_projection_matrix": cv_file.getNode("CAM_2_projection_matrix").mat(),
        "disparity_to_depth_matrix": cv_file.getNode("disparity_to_depth_matrix").mat(),
        "CAM_1_roi": cv_file.getNode("CAM_1_roi").mat(),
        "CAM_2_roi": cv_file.getNode("CAM_2_roi").mat(),
    }

    cv_file.release()
    return stereo_maps

In [8]:
# load intrinsics and extrinsics

with open(f"{root}/output/intrinsic_params.json", "r") as f:
    intrinsic_params = json.load(f)
    
with open(f"{root}/output/1_stereo_params.json", "r") as f:
    stereo_params = json.load(f)
    
# necessary parameters
K_cam1_undistort = np.array(intrinsic_params['CAM_1']['K_undistort'])
D_cam1_undistort = np.array(intrinsic_params['CAM_1']['D'])

K_cam2_undistort = np.array(intrinsic_params['CAM_2']['K_undistort'])
D_cam2_undistort = np.array(intrinsic_params['CAM_2']['D'])

stereo_maps = load_stereo_a_maps()
x_map_cam1 = stereo_maps['CAM_1_map_x']
y_map_cam1 = stereo_maps['CAM_1_map_y']

print(f"Map shapes: {x_map_cam1.shape}, {y_map_cam1.shape}")


P_cam1 = np.array(stereo_maps['CAM_1_projection_matrix'])
P_cam2 = np.array(stereo_maps['CAM_2_projection_matrix'])

x_map_cam2 = stereo_maps['CAM_2_map_x']
y_map_cam2 = stereo_maps['CAM_2_map_y']

print(f"Map shapes: {x_map_cam2.shape}, {y_map_cam2.shape}")



Map shapes: (2160, 3840), (2160, 3840)
Map shapes: (2160, 3840), (2160, 3840)


In [47]:
# TODO: get 2D coordinates from YOLO output
# NOTE: measure = pixels
cam1_2d = (cam1_element['x'], cam1_element['y'])
cam2_2d = (cam2_element['x'], cam2_element['y'])

# TODO: undirtort the points
# P to normalize the points into pixel coordinates
undistorted_point_camera_1 = cv2.undistortPoints(cam1_2d, K_cam1_undistort, D_cam1_undistort, P=K_cam1_undistort)
undistorted_point_camera_2 = cv2.undistortPoints(cam2_2d, K_cam2_undistort, D_cam2_undistort, P=K_cam2_undistort)
print(f"cam 1 undistorted: {undistorted_point_camera_1}")
print(f"cam 2 undistorted: {undistorted_point_camera_2}")

# TODO: projection matrix
P1 = np.dot(K_cam1_undistort, np.hstack((np.eye(3), np.zeros((3, 1)))))  # Camera 1's projection matrix
R2 = np.array(stereo_params['STEREO_A']['rotation_matrix'])  # Rotation matrix from stereo extrinsics
T2 = np.array(stereo_params['STEREO_A']['translation_vector'])  # Translation vector from stereo extrinsics
P2 = np.dot(K_cam2_undistort, np.hstack((R2, T2)))  # Camera 2's projection matrix

# TODO: triangulate
points_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistorted_point_camera_1, undistorted_point_camera_2)

# TODO: convert to 3D points
points_3d = points_3d_homogeneous[:3] / points_3d_homogeneous[3]
print(f"points 3d: {points_3d}")


cam 1 undistorted: [[[2563.13979095 1051.43508045]]]
cam 2 undistorted: [[[261.67304207 441.80245944]]]
points 3d: [[37252.63204383]
 [ 9925.9213036 ]
 [96443.29698413]]


In [49]:
threshold = 15  # threshold in pixels

# Use undistorted points as returned in pixel coordinates
undistorted_point_camera_1_pixels = undistorted_point_camera_1  
undistorted_point_camera_2_pixels = undistorted_point_camera_2  

# Use projection matrices P1 and P2 as computed (no scaling)
# Triangulation produced points_3d in world units (mm) but P1/P2 map from mm to pixels

# Reproject 3D points (in mm) back to 2D
reprojected_point_1 = np.dot(P1, np.vstack((points_3d, np.ones((1, points_3d.shape[1])))))
reprojected_point_2 = np.dot(P2, np.vstack((points_3d, np.ones((1, points_3d.shape[1])))))

reprojected_point_1 /= reprojected_point_1[2]
reprojected_point_2 /= reprojected_point_2[2]

error1 = np.linalg.norm(reprojected_point_1[:2] - undistorted_point_camera_1_pixels)
error2 = np.linalg.norm(reprojected_point_2[:2] - undistorted_point_camera_2_pixels)
print(f"error 1: {error1}")
print(f"error 2: {error2}")

if error1 > threshold or error2 > threshold:
    print("Anomaly detected!")

error 1: 2174.1676991445015
error 2: 613.7843033361829
Anomaly detected!
